# Question 1

In this question we will:

- Design and implement a feature that can be used to creates a feature map sensitive to "greenness".
- Utilize thresholding techniques of this "greenness" feature to obtain a segmentation of green leaves in images with a cluttered background.
- Implement evaluation metrics to measure the quality of a segmentation produced by the thresholding
- Use the metrics to evaluate the segmentation algoirthm's perormance on a dataset of images of green leaves on a cluttered background.

# Step 1: Write your Segmentation Algoirthm

Write a function that segments a leaf image, and returns a binary (`dtype='bool'`) image representing the segmentation.  Your algorithm must be based on thresholding.  Determine a metric that can be used to meaure the "greenness" of the colour of a given pixel.  Your algoirthm should create a "feature map" by computing this feature for each pixel, thus creating an "image" where each pixel's "intensity" is the value of the "greenness" feature.   Then use a thresholding method of your choice to segment the image's green regions.    You should also consider whether doing some region processing after segmentation can improve the results.  This function should return the segmenetion of the image as a binary image with a single connected component since you can take advantage of the fact that each image is known to contain only a single leaf.

_Hint: You'll need to be a bit creative when devising your solution -- no single technique from class is likely to give you a particularly good solution, and you may need to think of some tricks that were not explicitly covered in class.  However, you can get a good result with a fairly simple algorithm.  You'll also need to decide how to handle the fact that the input images are colour, although this shouldn't pose too much of a problem, in fact, it is an advantage!_

In [2]:
import skimage.util as util
import numpy as np
import os as os
from skimage import filters, morphology, measure
from scipy.ndimage import binary_fill_holes

def segleaf(I):
    '''
    Segment a leaf image.
    :param I: Color leaf image to segment.
    :return: Logical image where True pixels represent foreground (i.e. leaf pixels).
    '''
 
    I_float = util.img_as_float(I)

    R, G, B = I_float[:, :, 0], I_float[:, :, 1], I_float[:, :, 2]
    # Find excess green index (image is now 2D)
    excess_green = 2.0 * G - R - B
    # Renormalize image
    excess_green = (excess_green - excess_green.min()) / (excess_green.ptp() + 1e-8)
    t = filters.threshold_otsu(excess_green)
    mask = excess_green >= t

    # Clean image
    mask = morphology.remove_small_objects(mask, min_size=500)
    mask = morphology.binary_opening(mask, morphology.disk(3))
    mask = morphology.binary_closing(mask, morphology.disk(5))
    mask = binary_fill_holes(mask)

    # Label with 8-conectivity
    lbl = measure.label(mask, connectivity=2)
    if lbl.max() > 0:
        largest = np.argmax(np.bincount(lbl.ravel())[1:]) + 1
        mask = (lbl == largest)
    return mask

# Step 2: Implement Segmentation Performace Measures

Write functions to compute the Mean Squared Distance (MSD), Hausdorff Distance (HD) and Dice Similarity Coefficient (DSC) measures of segmentation quality.  

For MSD and HD, I suggest you reprsent boundaries by N-row, 2-column arrays where each row is the coordinate of one pixel on the region's boundary of the form [r,c], row first, then column.

In [3]:
from skimage.segmentation import find_boundaries
from scipy.ndimage import distance_transform_edt

def boundary_points(mask):
    b = find_boundaries(mask, mode='outer')
    coords = np.column_stack(np.nonzero(b))
    return coords, b

def dice(seg, gt):
    seg = seg.astype(bool)
    gt  = gt.astype(bool)
    inter = np.logical_and(seg, gt).sum()
    denom = seg.sum() + gt.sum()
    return (2.0 * inter) / (denom + 1e-8)

def msd_hd(seg, gt):
    A, Ab = boundary_points(seg)
    B, Bb = boundary_points(gt)
    if A.size == 0 or B.size == 0:
        return np.inf, np.inf
    dtB = distance_transform_edt(~Bb)
    dtA = distance_transform_edt(~Ab)
    dA = dtB[A[:, 0], A[:, 1]].astype(np.float64)
    dB = dtA[B[:, 0], B[:, 1]].astype(np.float64)
    hd = max(dA.max(initial=0.0), dB.max(initial=0.0))
    msd = 0.5 * (np.mean(dA**2) + np.mean(dB**2))
    return msd, hd


# Step 3: Write a Validation driver program.

Write code that segments each image (using the function in Step 1), and computes the MSD, HD, and DSC for each segmentation (using the functions in Step 2).  Print the MSD, HD, and DSC of each segmentation to the console as you perform it.  At the end, print the average and standard deviation of the DSC, the MSD, and the HD over all of the images.  Also print the percentage of regions that were "recognized" (see below).  Sample output is in the assignment description document.

The general approach should be, for each input image:

* load the image and it's ground truth (use the provided leaf image dataset, described in section 2.2. of the assignment PDF) -- a .csv file is provided with the names of all images so that you can process the files in the same manner as Assignment 1, question 1)
* segment the input image using your function from Step 1
* extract the region boundary points from the segmented image and ground truth image; store them in Nx2 arrays as described above (see lecture notes Topic 6, slide 68 for an example on how to do this).
* Compute the MSD and the HD from the two sets of boundary points (using the appropriate functions in Step 2).
* Compute the DSC from the segmented image and the ground truth image (using the appropriate function from Step 2).
* Determine whether the leaf was "recognized" (a leaf is recognized if it's DSC is greater than 0.6).
* Print the MSD, HD, and DSC to the console (see sample output).

When finished processing each image, don't forget to print the average and standard deviation of the DSC for all images, and the percentage of images where the leaf was "recognized".

_Feel free to define additional helper functions for your program if you think it will help._

In [ ]:
#### Validate ####

# Paths for folders -- original and ground truth images
images_path = os.path.join('.', 'images')
gt_path = os.path.join('.', 'groundtruth')

# TODO Change the I/O to use a CSV file like in assignment 0.


# Iterate over all files in the original images folder
for root, dirs, files in os.walk(images_path):
    for filename in files:
        # ignore files that are not PNG files.
        if filename[-4:] != '.png':
            continue
            
        # concatenate variable root with filename to get the path to an input file.



from skimage.io import imread
from skimage.color import rgb2gray

def load_gt(path):
    G = imread(path)
    Gg = (G.astype(np.float32) / (255.0 if G.dtype.kind in 'ui' else 1.0))
    gt = Gg > 0.5
    return gt

def evaluate(images_path='./images', gt_path='./groundtruth'):
    files = sorted([f for f in os.listdir(images_path) if f.lower().endswith('.png')])
    results = []
    for fname in files:
        ipath = os.path.join(images_path, fname)
        gpath = os.path.join(gt_path, fname)
        if not os.path.exists(gpath):
            print(f'Skipping {fname}: missing GT')
            continue
        I = imread(ipath)
        seg = segleaf(I)
        gt = load_gt(gpath)
        dsc = dice(seg, gt)
        msd, hd = msd_hd(seg, gt)
        recognized = dsc > 0.6
        results.append((fname, dsc, msd, hd, recognized, I, seg, gt))
        print(f'{fname} | DSC={dsc:.4f} | MSD={msd:.3f} | HD={hd:.3f} | recognized={recognized}')
    if not results:
        print('No images evaluated.')
        return results
    dscs = np.array([x[1] for x in results], dtype=float)
    msds = np.array([x[2] for x in results], dtype=float)
    hds  = np.array([x[3] for x in results], dtype=float)
    recs = np.array([x[4] for x in results], dtype=bool)
    print('\nOverall:')
    print(f'DSC mean={dscs.mean():.4f} std={dscs.std(ddof=1) if len(dscs)>1 else 0:.4f}')
    print(f'MSD mean={msds.mean():.3f} std={msds.std(ddof=1) if len(msds)>1 else 0:.3f}')
    print(f'HD  mean={hds.mean():.3f} std={hds.std(ddof=1) if len(hds)>1 else 0:.3f}')
    print(f'Recognized: {recs.mean()*100:.1f}%')
    return results

# Run evaluation
# results = evaluate('./images', './groundtruth')


# Step 4:  Display Examples

Choose one input image where your algoirthm performed very well.  Choose another image where the algorithm did not perform well.  Display the two original images with the segmentation superimposed on top (There is an example in the lecture notes -- last slide, Topic 6 -- showing how to do this).  Also display the same two image's ground truth with the segmentation superimposed on top.    Title the images to indicate which is the "good" example, and which is the "bad" example.


In [5]:
import matplotlib.pyplot as plt

%matplotlib inline



# Step 5: A time for reflection.

### Answer the following questions right here in this block.

1. In a few sentences, briefly explain what your segmentation algorithm from Step 1 does and how it works.  

	_Your answer:_  

2. Consider your example "good" result.  What, if anything, about your algoirthm is preventing you from getting a better result with this image?  If you weren't able to get any results, leave this blank, or explain what was preventing you from getting a result.

	_Your answer:_  

3. Consider your example "bad" result.  What is it about your algoirthm caused the poor performance?   If you weren't able to get any results, leave this blank.

	_Your answer:_  
